In [ ]:
import os 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn.preprocessing
import sklearn.metrics

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

import pymc3 as pm
import arviz as az

import voteestimator


In [ ]:
class AnalysisSetCreator:
    
    def __init__(self, votesmodel='Meindertsma'):

        votesmodels = {'Meindertsma': voteestimator.MeindertsmaVotesEstimator(),
                      'Exponential': voteestimator.ExponentialVotesEstimator()
                      }
        self.votesmodel = votesmodels[votesmodel]
    
    def _combine_data(self, filefolder):
        self.notering = pd.read_parquet(os.path.join(filefolder, 'notering.parquet'))
        self.song = pd.read_parquet(os.path.join(filefolder, 'song.parquet'))
        self.songartist = pd.read_parquet(os.path.join(filefolder, 'songartist.parquet'))
        self.artist = (pd.read_parquet(os.path.join(filefolder, 'artist.parquet')) # TODO: This should not happen here
                          .pipe(self._artist_features)
                        )
        
        df = (self.notering.merge(self.song, left_on='SongID', right_index=True)
                           .merge(self.songartist.reset_index())
                           .merge(self.artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
             )
        return df
    
    def _read_stemperiodes(self, path=os.path.join('Data', 'EindeStemperiode.xlsx')):
        einde_stemperiode = (pd.read_excel(path, engine='openpyxl')  # openpyxl does support xlsx
                               .dropna(subset=['EindeStemperiode'])
                               .drop(columns=['Bron'])
                               .sort_values('EindeStemperiode')
                            )
        return einde_stemperiode
    

    def _check_passed_away_during_top2000(self, df, top2000_stemperiodes):
        first_stemperiode = top2000_stemperiodes['EindeStemperiode'].min()
        relevant_date_of_death = first_stemperiode + pd.Timedelta('365 days')
        df['IsOverleden'] = df['Overlijdensdatum'].ge(relevant_date_of_death)
        return df
    
    def _find_next_top2000_after_death(self, df, top2000_stemperiodes):
        not_passed_away_during_top_2000 = df[~df['IsOverleden']].copy()
        passed_away_during_top2000 = (df.loc[df['IsOverleden']]
                                      .sort_values('Overlijdensdatum')
                                      .reset_index()
                                     )

        passed_away_during_top2000 = (pd.merge_asof(passed_away_during_top2000, top2000_stemperiodes,
                                                   left_on='Overlijdensdatum', right_on='EindeStemperiode', direction='forward')
                                     .set_index('ArtistID')
                                     )
        df = pd.concat([not_passed_away_during_top_2000, passed_away_during_top2000], sort=False)
        return df


    def _artist_features(self, df):
        einde_stemperiode = self._read_stemperiodes()
        df = (df                                          
                .pipe(self._check_passed_away_during_top2000, einde_stemperiode)
                .pipe(self._find_next_top2000_after_death, einde_stemperiode)
                .assign(AgePassing = lambda df: df['Overlijdensdatum'].sub(df['Geboortedatum']).dt.days / 365.25,
                        PassingTooEarly = lambda df: df['AgePassing'].sub(80).mul(-1).clip(lower=0),
                        IsDutch = lambda df: df['IsDutch'].astype(int),
                        )
             )
        return df
    
    def _rank_features(self, df):
        return df.assign(PctVotes = lambda df: df['Rank'].apply(self.votesmodel.percentage_of_votes))
    
    
    def _normalize_by_years_before_death(self, df, years_to_normalize=1):        
        mi = pd.MultiIndex.from_product([df.query('IsOverleden')['SongID'].unique(),
                                         df.query('IsOverleden')['YearsSinceOverlijden'].unique(),],
                                        names=['SongID', 'YearsSinceOverlijden'])
        votes_before_death = (pd.DataFrame(index=mi)
                              .join(self.songartist)
                              .join(df.set_index(['SongID', 'YearsSinceOverlijden', 'ArtistID'])[['Year', 'PctVotes']])
                              .join(self.artist[['JaarTop2000']])
                              .join(self.song[['YearMade']])
                              .assign(YearTop2000 = lambda df: df['JaarTop2000'].add(df.index.get_level_values('YearsSinceOverlijden')),
                                      PctVotes = lambda df: np.where(df['YearTop2000'].gt(df['YearMade']) & df['YearTop2000'].le(df['Year'].max()),
                                                             df['PctVotes'].fillna(self.votesmodel.lower_than_2000), np.nan)
                                     )
                             ['PctVotes']
                             .unstack('YearsSinceOverlijden')
                             .loc[:, range(-years_to_normalize, 0)]
                             .mean(axis='columns')
                             .rename('PctVotesBeforeDeath')
                             .reset_index()
                             )
        
        df = df.merge(votes_before_death, how='left')
        return df

    
    def _song_features(self, df):
        
        df = (df.assign(NrArtists = lambda df: df.groupby(['SongID', 'Year'])['Rank'].transform('count'),
                        YearsSinceOverlijden = lambda df: df['Year'].sub(df['JaarTop2000']),
                       )
                .pipe(self._normalize_by_years_before_death)
             )
        return df
    
    def _song_features_after_passing(self, df):
        df = (df.assign(NrsBeforeDeath = lambda df: df.groupby('ArtistID')['ArtistID'].transform('count'),
                        PopularityWithinArtist = lambda df: df.groupby('ArtistID')['PctVotesBeforeDeath'].apply(lambda v: v.div(v.mean())),
                        LogSongPopularityWithinArtist = lambda df: np.log10(df['PopularityWithinArtist']),
                        RecencyWithinArtist = lambda df: df.groupby('ArtistID')['YearMade'].apply(lambda v: v.sub(v.min()).div(v.max() - v.min())),
                        YearsBeforeDeath = lambda df: df['YearMade'].sub(df['JaarTop2000']),
                        Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                        MultiplePerformers = lambda df: df['NrArtists'].gt(1).astype(int),
                        JarenGeleden = lambda df: df['JaarTop2000'].sub(df['JaarTop2000'].max()),
                        )
             )
        return df
    
    def create_analysis_set(self, filefolder):
        df = (self._combine_data(filefolder)
                  .pipe(self._rank_features)
                  .pipe(self._song_features)
                  .query('YearsSinceOverlijden == 0')
                  .query(f'PctVotesBeforeDeath > {self.votesmodel.lower_than_2000}')
                  .pipe(self._song_features_after_passing)
             )
        return df
    
    def create_artist_set(self, filefolder):
        df = self.create_analysis_set(filefolder)
        df_artist = (df.groupby('ArtistID')
                        .agg({'PctVotes': 'sum',
                              'PctVotesBeforeDeath': 'sum',
                               'YearMade': 'last'
                            }
                            )
                        .join(self.artist[['Name', 'IsDutch', 'AgePassing', 'JaarTop2000', 'Overlijdensdatum', 'EindeStemperiode']])
                        .assign(DaysToStemperiode = lambda df: df['Overlijdensdatum'].sub(df['EindeStemperiode']).dt.days,
                                YearsSinceLastHit = lambda df: df['JaarTop2000'].sub(df['YearMade']),
                                LogPopularity = lambda df: np.log10(df['PctVotesBeforeDeath']),
                                LogPopularityNorm = lambda df: df['LogPopularity'].sub(df['LogPopularity'].median()),
                                Boost = lambda df: df['PctVotes'].div(df['PctVotesBeforeDeath']),
                                LogBoost = lambda df: np.log(df['Boost']),
                                )
                    )
        return df_artist
    
    def create_full_feature_set(self, filefolder):
        df = self.create_analysis_set(filefolder)
        df_artist = self.create_artist_set(filefolder)#.pipe(self._artist_features)
        full_set = (df.merge(df_artist, left_on='ArtistID', right_index=True, suffixes=('Song', 'Artist'))
                      .assign(
                              SongRelativeBoost = lambda df: df['BoostSong'].div(df['BoostArtist']),
                              LogRelativeBoost = lambda df: np.log2(df['SongRelativeBoost']),
                              LogBoost = lambda df: np.log(df['BoostSong']),
                             )
           )
        return full_set

a = AnalysisSetCreator()
df_song = a.create_analysis_set('Data')
df_artist = a.create_artist_set('Data')
df = a.create_full_feature_set('Data')

## Full out modelling

In [ ]:
features = ['JarenGeleden',
 'YearMadeSong',
 'AgePassingArtist',
 'NrArtists',
 'PctVotesBeforeDeathSong',
 'NrsBeforeDeath',
 'PopularityWithinArtist',
'LogSongPopularityWithinArtist',
'DaysToStemperiode',
 'LogPopularity',
 'IsDutchArtist'
]

In [ ]:
from sklearn.model_selection import train_test_split
X = df[features].assign(IsDutchArtist = lambda df: df['IsDutchArtist'].astype(int))
# X_train, X_test, y_train, y_test = train_test_split(X, df['LogBoost'], random_state=123)

In [ ]:
df['JarenGeleden'].value_counts(normalize=True).sort_index().cumsum()

In [ ]:
def split_train_test(X, y, jaren_geleden_split=-4):
    train_set = X['JarenGeleden'].le(jaren_geleden_split)
    X_train = X[train_set].copy()
    y_train = y[train_set].copy()
    X_test = X[~train_set].copy()
    y_test = y[~train_set].copy()
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_train_test(X, df['LogBoost'])

In [ ]:
df_train, df_test, y_train, y_test = split_train_test(df, df['LogBoost'])

# Dummy Classifier

In [ ]:
import sklearn.dummy

In [ ]:
dummy = sklearn.dummy.DummyRegressor(strategy='median')
dummy.fit(X_train, y_train)

In [ ]:
y_pred_train = dummy.predict(X_train)
sklearn.metrics.mean_absolute_error(y_train, y_pred_train)

In [ ]:
y_pred = dummy.predict(X_test)
sklearn.metrics.mean_absolute_error(y_test, y_pred)

In [ ]:
DUMMY_SCORE = sklearn.metrics.mean_absolute_error(y_test, y_pred)
model_results = {}

# TPOT

In [ ]:
from tpot import TPOTRegressor
tpot = TPOTRegressor(
    random_state=123,
    verbosity=2,
    n_jobs=-1,
    scoring='neg_mean_absolute_error',
    template='Transformer-Selector-Regressor',
    warm_start=True,
    memory='auto',
    early_stop=10
)
tpot.fit(X_train, y_train)

In [ ]:
# tpot.fit(X_train, y_train)

In [ ]:
y_pred = tpot.predict(X_test)
test_score_tpot = sklearn.metrics.mean_absolute_error(y_test, y_pred)
display(test_score_tpot)

In [ ]:
model_results['TPOT'] = test_score_tpot

In [ ]:
from sklearn.inspection import plot_partial_dependence
clf = tpot.fitted_pipeline_
fig, axes = plt.subplots(11, 1, figsize=(4, 16))
plot_partial_dependence(clf, X_train, X_train.columns, ax=axes)
plt.tight_layout()
# plot_partial_dependence(clf, X_train, [0, 1])

In [ ]:
yhat = tpot.predict(X_test)
plt.scatter(yhat, y_test)
plt.plot([0, 2], [0, 2], 'k--')

In [ ]:
tpot.score(X_train, y_train)

In [ ]:
tpot.score(X_test, y_test)

In [ ]:
yhat = tpot.predict(X_train)
plt.scatter(yhat, y_train)
plt.plot([0, 2], [0, 2], 'k--')

# Unpooled model

In [ ]:
RANDOM_SEED =42

In [ ]:
def unpooled_model_factory(X, y):
    coords = {"obs_id": np.arange(X.shape[0])}
    with pm.Model(coords=coords) as model:
        days_to_stemperiode = pm.Data('days_to_stemperiode', X['DaysToStemperiode'], dims='obs_id')
        logpopularity = pm.Data('logpopularity', X['LogPopularityNorm'], dims='obs_id')
        jaren_geleden = pm.Data("jaren_geleden", X['JarenGeleden'], dims='obs_id')
        passing_too_early = pm.Data('passing_too_early', X['PassingTooEarly'], dims='obs_id')
        is_dutch = pm.Data('is_dutch', X['IsDutchArtist'], dims='obs_id')

        multiple_performers = pm.Data('multiple_performers', X['MultiplePerformers'], dims="obs_id")
        popularity_within_oeuvre = pm.Data('popularity_within_oeuvre', X['LogSongPopularityWithinArtist'], dims="obs_id")

        # Hyperpriors:
        a = pm.Normal("a", mu=0, sigma=2.0)

        recency_effect_exponent = pm.Normal('recency_effect_exponent', mu=-1.5,sigma=1)
        max_recency_effect = pm.Normal('max_recency_effect', mu=2, sigma=2)
        effect_popularity = pm.Normal('effect_popularity', mu=0, sigma=2)
        history_effect = pm.Normal('history_effect', mu=0, sigma=1)
        age_passing_effect = pm.Normal('age_passing_effect', mu=0, sigma=1)
        is_dutch_effect = pm.Normal('is_dutch_effect', mu=0, sigma=2)

        # Expected value per artist:
        mu_artist = (a
                     + logpopularity * effect_popularity
                     # The correction of subtracting the minimum value is important for two reasons:
                     # 1. Since it fixes the minimum value at 1, it breaks the degeneracy with _a_, which makes sampling much more stable
                     # 2. It allows for much easier interpretation
                     + (np.exp(10**recency_effect_exponent * days_to_stemperiode)- np.exp(10**recency_effect_exponent * -365))* max_recency_effect
                     + jaren_geleden * history_effect
                     + passing_too_early * age_passing_effect
                     + is_dutch * is_dutch_effect
                    )

        sharing_effect = pm.Normal('sharing_effect', mu=0, sigma=2.0)
        within_oeuvre_effect = pm.Normal('within_oeuvre_effect', mu=0, sigma=2.0)
        theta = (mu_artist
                 + multiple_performers * sharing_effect
                 + popularity_within_oeuvre * within_oeuvre_effect
                )
        # Model error:
        sigma = pm.Exponential("sigma", 1.0)

        y_like = pm.Normal("y_like", theta, sigma=sigma, observed=y, dims="obs_id")

        return model

In [ ]:
with unpooled_model_factory(X=df_train,
                   y=y_train,
                   ) as multilevel_noncentered_model:
    display(pm.model_to_graphviz(multilevel_noncentered_model))
    unpooled_model_idata = pm.sample(10000, tune=3000, return_inferencedata=True, random_seed=RANDOM_SEED, target_accept=0.95)

In [ ]:
with unpooled_model_factory(X=df_test,
                   y=y_test,
                   ) as test_model:
    ppc_unpooled = pm.fast_sample_posterior_predictive(unpooled_model_idata.posterior,
                                         var_names=['y_like'],
                                        )


In [ ]:
y_pred_unpooled = np.median(ppc_unpooled['y_like'], axis=0)

In [ ]:
pd.DataFrame(ppc_unpooled['y_like']).median().sub(y_test.values).abs().agg(['mean', 'sem'])

In [ ]:
model_results['Unpooled'] = sklearn.metrics.mean_absolute_error(y_test, y_pred_unpooled)

# Partially pooled model

In [ ]:
import pymc3 as pm
import arviz as az

In [ ]:
RANDOM_SEED = 42
def model_factory(X, y):
    passed_away_artists = X['ArtistID'].unique()
    artist_lookup = dict(zip(passed_away_artists, range(len(passed_away_artists))))
    artist_vals = X['ArtistID'].replace(artist_lookup).values
    artist_model = (X.assign(ArtistIDModel = lambda df: X['ArtistID'].map(artist_lookup),
                             )
                .sort_values('ArtistIDModel')
                .drop_duplicates(['ArtistIDModel'])
               )
    
    
    coords = {"obs_id": np.arange(X.shape[0]),
              'Artist': range(len(passed_away_artists))
         }
    with pm.Model(coords=coords) as model:
        artist_idx = pm.Data("artist_idx", artist_vals, dims="obs_id")
        days_to_stemperiode = pm.Data('days_to_stemperiode', artist_model['DaysToStemperiode'], dims='Artist')
        logpopularity = pm.Data('logpopularity', artist_model['LogPopularityNorm'], dims='Artist')
        jaren_geleden = pm.Data("jaren_geleden", artist_model['JarenGeleden'], dims='Artist')
        passing_too_early = pm.Data('passing_too_early', artist_model['PassingTooEarly'], dims='Artist')
        is_dutch = pm.Data('is_dutch', artist_model['IsDutchArtist'], dims='Artist')

        multiple_performers = pm.Data('multiple_performers', X['MultiplePerformers'], dims="obs_id")
        popularity_within_oeuvre = pm.Data('popularity_within_oeuvre', X['LogSongPopularityWithinArtist'], dims="obs_id")

        # Hyperpriors:
        a = pm.Normal("a", mu=0, sigma=2.0)
        sigma_a = pm.Exponential("sigma_a", 1.0)

        recency_effect_exponent = pm.Normal('recency_effect_exponent', mu=-1.5,sigma=1)
        max_recency_effect = pm.Normal('max_recency_effect', mu=2, sigma=2)
        effect_popularity = pm.Normal('effect_popularity', mu=0, sigma=2)
        history_effect = pm.Normal('history_effect', mu=0, sigma=1)
        age_passing_effect = pm.Normal('age_passing_effect', mu=0, sigma=1)
        is_dutch_effect = pm.Normal('is_dutch_effect', mu=0, sigma=2)

        # Expected value per artist:
        mu_artist = (a
                     + logpopularity * effect_popularity
                     # The correction of subtracting the minimum value is important for two reasons:
                     # 1. Since it fixes the minimum value at 1, it breaks the degeneracy with _a_, which makes sampling much more stable
                     # 2. It allows for much easier interpretation
                     + (np.exp(10**recency_effect_exponent * days_to_stemperiode)- np.exp(10**recency_effect_exponent * -365))* max_recency_effect
                     + jaren_geleden * history_effect
                     + passing_too_early * age_passing_effect
                     + is_dutch * is_dutch_effect
                    )

        # This is the non-centered version of the model for a much more stable sampling
        # See https://twiecki.io/blog/2017/02/08/bayesian-hierchical-non-centered/ for more information
        mu_artist = pm.Deterministic("mu_artist", mu_artist, dims="Artist")
        za_artist = pm.Normal("za_artist", mu=0.0, sigma=1.0, dims='Artist')
        a_artist = pm.Deterministic("a_artist", mu_artist + za_artist * sigma_a, dims="Artist")
        sharing_effect = pm.Normal('sharing_effect', mu=0, sigma=2.0)
        within_oeuvre_effect = pm.Normal('within_oeuvre_effect', mu=0, sigma=2.0)
        theta = (a_artist[artist_idx]
                 + multiple_performers * sharing_effect
                 + popularity_within_oeuvre * within_oeuvre_effect
                )
        # Model error:
        sigma = pm.Exponential("sigma", 1.0)

        y_like = pm.Normal("y_like", theta, sigma=sigma, observed=y, dims="obs_id")

        return model

In [ ]:
with model_factory(X=df_train,
                   y=y_train,
                   ) as multilevel_noncentered_model:
    display(pm.model_to_graphviz(multilevel_noncentered_model))
    multilevel_noncentered_model_idata = pm.sample(10000, tune=3000, return_inferencedata=True, random_seed=RANDOM_SEED, target_accept=0.95)

In [ ]:
with model_factory(X=df_test,
                   y=y_test,
                   ) as test_model:
    # For newly passed artists, we do not know what za_artist should be
    ppc_multilevel = pm.fast_sample_posterior_predictive(multilevel_noncentered_model_idata.posterior.drop_vars(['mu_artist', 'a_artist', 'za_artist']),
                                        )


In [ ]:
az.concat(multilevel_noncentered_model_idata, az.from_dict(posterior_predictive=ppc_multilevel), inplace=True)

In [ ]:
np.abs(ppc_multilevel['y_like'].mean(axis=0) - y_test.values).mean()

In [ ]:
pd.DataFrame(ppc_multilevel['y_like']).describe()

In [ ]:
y_pred_multilevel = np.median(ppc_multilevel['y_like'], axis=0)
sklearn.metrics.mean_absolute_error(y_test, y_pred_multilevel)

In [ ]:
model_results['Partially pooled'] = sklearn.metrics.mean_absolute_error(y_test, y_pred_multilevel)

# Visualization

In [ ]:
y_pos = range(len(model_results), 0, -1)
plt.barh(y_pos, list(model_results.values()))
plt.gca().set_yticks(y_pos)
plt.gca().set_yticklabels(labels=list(model_results.keys()))
plt.gca().axvline(DUMMY_SCORE, c='k', ls='--', label='Random')
plt.xlabel('Mean absolute error\n(lower is better)')
plt.legend()
plt.show()

In [ ]:
model_compare  = az.compare({'Multilevel': multilevel_noncentered_model_idata,
                             'Unpooled': unpooled_model_idata},
                           )
display(model_compare)
az.plot_compare(model_compare, insample_dev=False)

In [ ]:
multilevel_noncentered_model_idata

In [ ]:
az.loo_pit(idata=multilevel_noncentered_model_idata, y=y_train.values, y_hat=ppc_multilevel['y_like'].T)

In [ ]:
axes = az.plot_posterior(multilevel_noncentered_model_idata.posterior, var_names=['~za_artist', '~mu_artist', '~a_artist', '~sigma_a'], hdi_prob='hide')
az.plot_posterior(unpooled_model_idata.posterior, var_names=['~sigma_a'], ax=axes, c='black', hdi_prob='hide')
plt.legend()
plt.show()